In [1]:
data_path = "../../../data/"

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

# TODO

  * [x] фильтрация
  * [x] запись для бэкенда
  * [ ] функция получения доли импортных товаров
    * [ ] регрессия и прогноз
  * [ ] функция получения баланса товара у ИНН
  * [ ] частотный анализ и выделение сезонности

# [+] Загрузка данных и предварительная фильтрация

## Ввод

In [3]:
df_intro = pd.read_csv(data_path + "Данные о вводе товаров в оборот с 2021-11-22 по 2022-11-21.csv").dropna()

In [4]:
print(df_intro.columns)
df_intro.head(5)
# 'gtin' - Глобальный номер товарной продукции,'prid' - , 'operation_type' - тип ввода
# 1. РФ
# 2. Импорт из-за
# пределов ЕАЭС
# 3. Маркировка
# остатков
# 4. Ввоз из ЕАЭС
# 5. Перемаркировка
# 6. Принято от
# физического лица
# 7. Возвраты
# 'cnt' - кол-во упаковок

Index(['dt', 'inn', 'gtin', 'prid', 'operation_type', 'cnt'], dtype='object')


,dt,inn,gtin,prid,operation_type,cnt
0,2021-11-22,F0C677A146366202906368AF9FCEC188,C238E7C33B56BA911F576777316EC9F2,F0C677A146366202906368AF9FCEC188,Импорт. Не ЕАЭС,6
1,2021-11-22,F0DC262C40FBA363CFCEF8005E4C6140,4D3423957ECDC40F70E37EB75A6A2D3E,F0DC262C40FBA363CFCEF8005E4C6140,Перемаркировка,10
2,2021-11-22,32D845A9CC2A5BFA9F77CAE0E9A905A5,F1931263F03E1D89A43A3633AAF9FB84,32D845A9CC2A5BFA9F77CAE0E9A905A5,Импорт. Не ЕАЭС,600
3,2021-11-22,CE816FDF82B121C67CAE9B79203B8BA1,0374D8CAC82E9F0D47C95FA13D68C7C6,CE816FDF82B121C67CAE9B79203B8BA1,Импорт. Не ЕАЭС,1860
4,2021-11-22,E34F3F6C9E49FE46C87D067306AAC29B,BBC31CA374A30B34CAFBFD027888A73D,E34F3F6C9E49FE46C87D067306AAC29B,РФ,1


## Вывод

In [4]:
# грузится дольше всех, около 3 минут, 11 Гб
# df_remove = pd.read_csv(data_path + "Данные о выводе товаров из оборота с 2021-11-22 по 2022-11-21.csv").dropna()
# tqdm context
chunksize = 100000
df_remove = pd.DataFrame()
length = 52100000
filename = data_path + "Данные о выводе товаров из оборота с 2021-11-22 по 2022-11-21.csv"
frames = []
with tqdm(total=length, desc="chunks read: ") as bar:
    # enumerate chunks read without low_memory (it is massive for pandas to precisely assign dtypes)
    for i, chunk in enumerate(
        pd.read_csv( filename, chunksize=chunksize, low_memory=False)):
        
        # print the chunk number
        # print(i)
        
        # append it to df
        # df_remove = df_remove.append(other=chunk)
        frames.append(chunk.dropna())
        
        # update tqdm progress bar
        bar.update(chunksize)
df_remove = pd.concat(frames)

chunks read:   0%|          | 0/52100000 [00:00<?, ?it/s]

In [7]:
print(df_remove.columns)
df_remove.head(5)

Index(['dt', 'gtin', 'prid', 'inn', 'id_sp_', 'type_operation', 'price',
       'cnt'],
      dtype='object')


,dt,gtin,prid,inn,id_sp_,type_operation,price,cnt
0,2021-11-22,5A3E5F7B2D093D1D6CB3CF93BA9AC8A6,DC7432556B58D454B4E37F0F0D1E663D,B035ED5D308BC70E5EA9342B6DBF5810,19485F700068E6A9A8593AFDE9803638,Продажа конечному потребителю в точке продаж,3500,1
1,2021-11-22,9A2202A48FC5E462C44B78B0C79E1DBD,D18B1B470E57871FAB5B4E949E256BB4,DD486FED86564603D2CD7CF9EED2C458,C2969A4C8A40C12902EAD4936E303045,Продажа конечному потребителю в точке продаж,56714,1
2,2021-11-22,877ABE9DE14E0B510DFC8DE63EEFA817,62B5EAEF59E6693F44B1074C1140BF45,62B5EAEF59E6693F44B1074C1140BF45,5637334F301773E28AE8730713E6D5F7,Продажа конечному потребителю в точке продаж,74314,1
3,2021-11-22,B004F889F17F8AB365C494B3D0D06E1A,D7326D87A37D73AF19AF53668A7116D6,D7326D87A37D73AF19AF53668A7116D6,0FF56DEC16050D3B6372F2CB1BD6DFCB,Продажа конечному потребителю в точке продаж,4999,1
4,2021-11-22,E1C0503EA9201D56D7173CB4CA4A6AFA,50955BB0B97298EEAEB1F6CF9712F37A,39FFA592BE77CA7DE37C65C1B8423E8C,9E660AA1DE428BA88D1942DB2824BDAB,Продажа конечному потребителю в точке продаж,29857,1


## Перемещения

In [5]:
df_transactions = pd.read_csv(data_path + "Данные о перемещениях товаров между участниками с 2021-11-22 по 2022-11-21.csv").dropna()

In [6]:
print(df_transactions.columns)
df_transactions.head(5)
# prid - ИНН владельца карточки товара

Index(['dt', 'gtin', 'prid', 'sender_inn', 'receiver_inn', 'cnt_moved'], dtype='object')


,dt,gtin,prid,sender_inn,receiver_inn,cnt_moved
0,2021-11-22,867FDFE9E03925792F329DEDBAA3955F,9246A2BE043136FC4B4A166BB0E25D3E,2E55013259E009F4FC1E5EFCF170EA8B,8D5F8E7D20AD9A8D0E8EE5769321D8C3,1
1,2021-11-22,4FB53B945150E362C307D60297066562,9246A2BE043136FC4B4A166BB0E25D3E,32B53198A919C559CB84DC0487947E80,EDE6F6D21300D96A9E207EA4F59321EF,1
2,2021-11-22,38BA9B93C5C987770AD698646D2FAB66,2DADC5DF60A9FC194A3388C1612CC967,3A87113AC25EFD88BD3407ACECFA3869,D02A2294DDF3BCF7218D6FEFDB35A906,1
3,2021-11-22,FA77213DA058666C6781164754CBC0EB,FFC432B34974A68B2845FBED9A7DEB29,C5CDC4F467B7CDA0F3E082BFFEDA9938,051C7D77FE590C5428DB57FD48C5DB9C,3
4,2021-11-22,E5635C538FE51D88504CE5CFF8BC1D2C,6E536425E534ABA7AB9FEA2ACA91A775,AE7F5C67298A67CACD2A193C9405524F,0D28C0E0C8ADAA3D9B55B557C71D8A8E,2


## Продукция, торговые точки, участники оборота

In [7]:
df_products = pd.read_csv(data_path + "Справочник продукции.csv").dropna()
# volume - вес единицы товара

df_retailers = pd.read_csv(data_path + "Справочник торговых точек.csv").dropna()
# id_sp - ID точки продажи, inn - ИНН организацииб, city_with_type - город

df_organizations = pd.read_csv(data_path + "Справочник участников оборота товаров.csv").dropna()
# ИНН и код региона регистрации

In [8]:
print(df_products.columns)
df_products.head()

Index(['gtin', 'inn', 'product_name', 'product_short_name', 'tnved', 'tnved10',
       'brand', 'country', 'volume'],
      dtype='object')


,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
11,48CA1A98E725707624CBE5CFDDF84E4B,4AC29B08BEEDD5ADDCC0CF04F6A467CF,562149120122CD61A62AF200F816A906,2868B842B46F754C088D89E130F89585,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,7285BF2EA4DCF84E476C270608C89CB2,РОССИЯ,5
15,2FDF6EF3AB7614C60E3791C784F9FE60,77F82E55113DC87D9DD6E67AC7C3B6A7,4A4EF609DBAD0413B3B15DC2EC077796,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
16,52B3E1D2885E2CF3562F1B4D0D610F24,77F82E55113DC87D9DD6E67AC7C3B6A7,7D7F778F3D80D3700E173CD4C8F56D05,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
17,9B56606CD087B76D6010BA684AB5FF84,77F82E55113DC87D9DD6E67AC7C3B6A7,B9349E4D15AF29F36C132CE0E47572FD,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
18,8E7DC313B08BC5A42E1B5D255C7FE518,77F82E55113DC87D9DD6E67AC7C3B6A7,41C10E52BA066845D21731CA380EC9A3,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100


In [9]:
print(df_retailers.columns)
df_retailers.head()

Index(['id_sp_', 'inn', 'region_code', 'city_with_type', 'city_fias_id',
       'postal_code'],
      dtype='object')


,id_sp_,inn,region_code,city_with_type,city_fias_id,postal_code
1,591942644AE25AE4DD26CF943BB97414,81AAC456284F455FC2B141E6E8F1A76C,66,г Екатеринбург,2763c110-cb8b-416a-9dac-ad28a55b4402,620027.0
2,A6E2261F16F84FF09E2B615EDB008943,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390026.0
3,DC3B647A089FA064DE61E0C778F5EE45,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390017.0
4,D2F125BAA947B6293CD055DA1DEEA87D,87C2FDB73C26C96266A0947C2BB714BF,62,г Рыбное,91d71d8c-2f1a-43c8-bc84-d78dc4245a1b,391112.0
5,E81FFF6826AA5F23C672F240B44CC492,123FAE5D025194921149E5B0A1E0DF72,74,г Челябинск,a376e68d-724a-4472-be7c-891bdb09ae32,454010.0


In [10]:
print(df_organizations.columns)
df_organizations.head()

Index(['inn', 'region_code'], dtype='object')


,inn,region_code
0,60DFB6F87ABF6232A5D97A1A827F8D88,1
1,2D13C279C7CEC509D21695F0268ECCEA,1
2,1705C73792D7A1A5ABDF1883FE851004,1
3,B4CC35BD7D8F69E665160F548EBE125D,23
4,2DB318BEA4A44280FA488A0B2AF36254,1


# [+] Запись фильтрованных для бекенда (для перезаписи перевести в code)

# Фильтрация

## [+] Товары, у которых не числовой volume

In [11]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64218 entries, 11 to 393466
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   gtin                64218 non-null  object
 1   inn                 64218 non-null  object
 2   product_name        64218 non-null  object
 3   product_short_name  64218 non-null  object
 4   tnved               64218 non-null  object
 5   tnved10             64218 non-null  object
 6   brand               64218 non-null  object
 7   country             64218 non-null  object
 8   volume              64218 non-null  object
dtypes: object(9)
memory usage: 4.9+ MB


In [18]:
# примерно 1/10 часть volume не цифровая (см распечатку 1 строки)
display(df_products[~df_products['volume'].str.isdigit()].head())
display(df_products[df_products['volume'].str.isdigit()].head())

,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
224,4E30BE4F9AD22F04F60786FAE7F61608,15BBDC0571C8EFC07FBF95BF060FE70A,7C2CD58E6EB5476F85D2DF86D19664C5,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,1DD155D79562007FC280B3779CC8D471,ШВЕЙЦАРИЯ,100мл
1081,524C79CDF6058ACAF71AC78BC55DC7C7,D18B1B470E57871FAB5B4E949E256BB4,A10C53C8ADEAB887FD45E1148CE5DA47,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,04009E8D507B872ABEA1063261897F25,ИСПАНИЯ,НЕ КЛАССИФИЦИРОВАНО
1386,85DA6DED98B4A8707F4036AE7DABCAD4,93E0A152D5F486EA13C9DEFA24625849,1CAE83BE2EC44B2B100247B861837C8F,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,80DDDF049ACD57149A4ECBFCA88B10A3,СОЕДИНЕННЫЕ ШТАТЫ,НЕ КЛАССИФИЦИРОВАНО
1387,8F6ADB15D8D0CF4A8569F95D29EF9DC2,93E0A152D5F486EA13C9DEFA24625849,2296C615582DA198EC954AFF290B7618,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,80DDDF049ACD57149A4ECBFCA88B10A3,СОЕДИНЕННЫЕ ШТАТЫ,НЕ КЛАССИФИЦИРОВАНО
1669,ACC30286ECE394F52B21AB1A0CCDD52A,4016FCC6FD2B65013CD5F3B7A5135705,BA71B727932B90A5FD8D7337C8D17433,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,831E88535C411B1B6F446B825D4A860D,ОБЪЕДИНЕННЫЕ АРАБСКИЕ ЭМИРАТЫ,НЕ КЛАССИФИЦИРОВАНО


,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
11,48CA1A98E725707624CBE5CFDDF84E4B,4AC29B08BEEDD5ADDCC0CF04F6A467CF,562149120122CD61A62AF200F816A906,2868B842B46F754C088D89E130F89585,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,7285BF2EA4DCF84E476C270608C89CB2,РОССИЯ,5
15,2FDF6EF3AB7614C60E3791C784F9FE60,77F82E55113DC87D9DD6E67AC7C3B6A7,4A4EF609DBAD0413B3B15DC2EC077796,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
16,52B3E1D2885E2CF3562F1B4D0D610F24,77F82E55113DC87D9DD6E67AC7C3B6A7,7D7F778F3D80D3700E173CD4C8F56D05,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
17,9B56606CD087B76D6010BA684AB5FF84,77F82E55113DC87D9DD6E67AC7C3B6A7,B9349E4D15AF29F36C132CE0E47572FD,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
18,8E7DC313B08BC5A42E1B5D255C7FE518,77F82E55113DC87D9DD6E67AC7C3B6A7,41C10E52BA066845D21731CA380EC9A3,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,E623CD7FCC7F5B673C8A3E9C5B233668,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100


# [+] Функции

## [+] Баланс товара

Параметры:
1) gtin
2) категория товара
3) ИНН организации
4) на какую дату

Выходные параметры:
1) приход
2) расход

In [33]:
df_products.loc[df_products['gtin'] == 'E5635C538FE51D88504CE5CFF8BC1D2C']

,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
290904,E5635C538FE51D88504CE5CFF8BC1D2C,FFF15059A12B639F2E0B2F5421D79688,AA3ED2C75EF3253BFE65F8704E7F4B2E,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,F70FDAE164B13C52166628BA69763AEF,РОССИЯ,NaN
290905,E5635C538FE51D88504CE5CFF8BC1D2C,6E536425E534ABA7AB9FEA2ACA91A775,AA3ED2C75EF3253BFE65F8704E7F4B2E,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,F70FDAE164B13C52166628BA69763AEF,РОССИЯ,NaN
290906,E5635C538FE51D88504CE5CFF8BC1D2C,3BA981B1FA774AB2935E52ACD11FBDC4,AA3ED2C75EF3253BFE65F8704E7F4B2E,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,F70FDAE164B13C52166628BA69763AEF,РОССИЯ,100


In [37]:
from pandas.api.types import is_number
df_products['volume']

0                          10
1         НЕ КЛАССИФИЦИРОВАНО
2                         100
3                         100
4                          50
                 ...         
393462                    100
393463                    100
393464                    100
393465                     75
393466                      6
Name: volume, Length: 393467, dtype: object

In [29]:
def on_market(dt:str, gtin:str, tnved:str = None, tnved10:str = None):
    """
    Функция подсчитывает кол-во товара с заданным gtin
    на рынке (разница между тем, сколько введено и тем,
    сколько выведено)
    
    Если задан tnved, то он используется вместо gtin и tnved10
    
    Если задан tnved10, то он используется вместо gtin и tnved
    
    Args:
        inn:str - хеш ИНН юрлица
        dt:str - дата в формате ISO ГГГГ-MM-ДД
        gtin:str - хеш GTIN
    """
    #TODO фильтрация по tnved10 и tnved
    # фильтрация по дате и gtin
    filtered_intro = df_intro[df_intro['gtin'] == gtin]
    filtered_intro = filtered_intro[filtered_intro['dt'] <= dt]
    
    filtered_removal = df_remove[df_remove['gtin'] == gtin]
    filtered_removal = filtered_removal[filtered_removal['dt'] <= dt]
    
    result = filtered_intro['cnt'].sum() - filtered_removal['cnt'].sum()
    
    return result

def inn_balance(inn:str, dt:str, gtin:str, tnved:str = None, tnved10:str = None):
    """
    Функция подсчитывает для юрлица inn
    приход и расход товаров с заданным gtin
    на момент dt от начала БД
    
    Если задан tnved, то он используется вместо gtin и tnved10
    
    Если задан tnved10, то он используется вместо gtin и tnved
    
    Args:
        inn:str - хеш ИНН юрлица
        dt:str - дата в формате ISO ГГГГ-MM-ДД
        gtin:str - хеш GTIN
    """
    #TODO фильтрация по tnved10 и tnved
    # фильтрация по вермени и коду товара
    filtered_transactions = df_transactions[df_transactions['gtin'] == gtin]
    filtered_transactions = filtered_transactions[filtered_transactions['dt'] <= dt]
    
    filtered_intro = df_intro[df_intro['gtin'] == gtin]
    filtered_intro = filtered_intro[filtered_intro['dt'] <= dt]
    
    filtered_removal = df_remove[df_remove['gtin'] == gtin]
    filtered_removal = filtered_removal[filtered_removal['dt'] <= dt]
    
    # приход
    positive_transactions = filtered_transactions[filtered_transactions['receiver_inn']==inn]
    positive_transactions_sum_cnt = positive_transactions['cnt_moved'].sum()
    intro_sum = filtered_intro[filtered_intro['inn'] == inn]['cnt'].sum()
    positive_sum_cnt = positive_transactions_sum_cnt + intro_sum
    # positive_sum_vol = positive_sum_cnt * df_products['volume'] # данные не подходят для расчёта физической размерности
    
    # расход
    negative_transactions = filtered_transactions[filtered_transactions['sender_inn']==inn]
    negative_transactions_sum_cnt = negative_transactions['cnt_moved'].sum()
    removal_sum = filtered_removal[filtered_removal['inn'] == inn]['cnt'].sum()
    negative_sum_cnt = negative_transactions_sum_cnt + removal_sum
    
    return (positive_sum_cnt, negative_sum_cnt)

def search_over

In [33]:
# тест
inn_balance(inn="E34F3F6C9E49FE46C87D067306AAC29B", dt="2022-12-01", gtin = "BBC31CA374A30B34CAFBFD027888A73D")

(16, 6)

In [32]:
# тест
on_market(dt="2022-12-01", gtin = "BBC31CA374A30B34CAFBFD027888A73D")

11

## [+] Доля импортных товаров

In [34]:
df_intro['operation_type'].unique()

array(['Импорт. Не ЕАЭС', 'Перемаркировка', 'РФ', 'Маркировка остатков',
       'Импорт. ЕАЭС', 'Возврат', 'Принято от физического лица'],
      dtype=object)

In [35]:
df_products['country'].unique()

array(['РОССИЯ', 'СОЕДИНЕННОЕ КОРОЛЕВСТВО', 'ШВЕЙЦАРИЯ',
       'СОЕДИНЕННЫЕ ШТАТЫ', 'ФРАНЦИЯ', 'ИСПАНИЯ', 'ГЕРМАНИЯ',
       'ОБЪЕДИНЕННЫЕ АРАБСКИЕ ЭМИРАТЫ', 'ИТАЛИЯ', 'ОМАН', 'ЯПОНИЯ',
       'НИДЕРЛАНДЫ', 'БАХРЕЙН', 'ЛАТВИЯ', 'КИТАЙ', 'ТУРЦИЯ', 'ИНДИЯ',
       'БЕЛАРУСЬ', 'ПОЛЬША', 'ШВЕЦИЯ', 'ТАЙВАНЬ (КИТАЙ)', 'ЛИВАН',
       'СЛОВАКИЯ', 'САУДОВСКАЯ АРАВИЯ', 'УКРАИНА', 'ГОНКОНГ', 'БЕЛЬГИЯ',
       'КАНАДА', 'ДАНИЯ', 'БОЛГАРИЯ', 'ИЗРАИЛЬ', 'КАЗАХСТАН', 'АНГИЛЬЯ',
       'МОНАКО', 'САН-МАРИНО', 'СЛОВЕНИЯ', 'ГРЕЦИЯ', 'ЧЕХИЯ', 'АРМЕНИЯ',
       'ТАИЛАНД', 'ИРЛАНДИЯ', 'МАРОККО', 'КОРЕЯ, РЕСПУБЛИКА', 'АВСТРАЛИЯ'],
      dtype=object)

In [44]:
countries_rf = ['РОССИЯ',]
countries_eaes = ['АРМЕНИЯ', 'БЕЛАРУСЬ', 'КАЗАХСТАН', ]
df_products[~df_products['country'].isin([*countries_eaes,*countries_rf])]['gtin'].to_list()

['2FDF6EF3AB7614C60E3791C784F9FE60',
 '52B3E1D2885E2CF3562F1B4D0D610F24',
 '9B56606CD087B76D6010BA684AB5FF84',
 '8E7DC313B08BC5A42E1B5D255C7FE518',
 '3F97C2E0A0B0528FEE208FC912487B4B',
 '7ED7B34D018E7587D72E63C3E280CF6E',
 'C9CB046B649F492D4A9D3C99869C27A4',
 '3C9D36E4BD49F76C74FBD8EF8F7D1AB7',
 '49A952DA2C2DE1DA47A03C219DF7A1A0',
 '9CFEEAE7748509C3CE38AAB49BF7BC28',
 '35BA224679641645505A66D01FF41714',
 '6EDCFB8C1111654C426C15D0FC02D3B9',
 'D3803AF6A1B89C70E12C4FA4C69A2438',
 '97D3BC82467877D4631E8DCD7742144D',
 '200D196E4288AF9D816FAF6440488152',
 'B0E7C7DE5824D038B36FE5420FF080E7',
 'E2E7985320FE770D11AB81C5E54EE2E2',
 'B6AF40C980A36D4A40C23A5C999364E5',
 '61B86F56EB8C19CEF2A1A1AA497AB763',
 '6E13A65A971F8DF8C524B7A37ECABE8D',
 '052DF3D6DB650C4F11F3F4417E845B9A',
 'B40C7790204A3F0BCF44722F6550EF40',
 '52D477071A4D4088BE7E1432ADF293B9',
 '9CDE9C95018A06C271B703C6E810E3ED',
 'CCB39C42B2BD4EC308E6ACDF9489E9F7',
 '49FB3109F451B62CDACD3E35953993AF',
 '4E30BE4F9AD22F04F60786FAE7F61608',
 

In [45]:
(df_intro['dt'] >= '2021-01-01') & (df_intro['dt'] <= '2021-01-02')

0         False
1         False
2         False
3         False
4         False
          ...  
740579    False
740580    False
740581    False
740582    False
740583    False
Name: dt, Length: 740584, dtype: bool

In [48]:
def import_vs_local(dt1:str, dt2:str, tnved:str, tnved10:str = None):
    """
    Расчёт кол-ва товаров из заданной категории tnved или tnved10
    производства РФ, EAЭС и импортных,
    тренд импортозамещения и его прогноз
    
    Возвращает cnt товаров, введённых в оборот ( российских, из остальных стран  ЕАЭС, иностранных )
    """

    
    # по  operation_type поля 'Импорт. Не ЕАЭС', 'Перемаркировка', 'РФ',
    # 'Маркировка остатков', 'Импорт. ЕАЭС', 'Возврат', 'Принято от физического лица'
    
    countries_rf = ['РОССИЯ',]
    countries_eaes = ['АРМЕНИЯ', 'БЕЛАРУСЬ', 'КАЗАХСТАН', ]
    # countries_foreign = not in [*country_foreign, *country_EAES]
    
    # фильтрация по tnved или tnved10
    if(tnved is not None):
        df_products_filtered = df_products[df_products['tnved']==tnved]
    elif(tnved10 is not None):
        df_products_filtered = df_products[df_products['tnved10'] == tnved10]
    else:
        raise "Не указан ни tnved, ни tnved10"
    
    # фильтрация по стране происходжения df_products_filtered['country']
    gtins_rf = df_products_filtered[df_products_filtered['country'].isin(countries_rf)]['gtin'].to_list() # список gtin из РФ
    gtins_eaes = df_products_filtered[df_products_filtered['country'].isin(countries_eaes)]['gtin'].to_list() # список gtin из остальных стран ЕАЭС
    gtins_foreign = df_products_filtered[~df_products_filtered['country'].isin([*countries_eaes,*countries_rf])]['gtin'].to_list() # список импортных guid
    
    # фильтрация ввода товара по дате
    dates_mask = (df_intro['dt'] >= dt1) & (df_intro['dt'] <= dt2)
    df_intro_filtered = df_intro[dates_mask]
    
    # фильтрация ввода товара по gtin страны происхождения
    df_intro_filtered_rf = df_intro_filtered[df_intro_filtered['gtin'].isin(gtins_rf)]
    df_intro_filtered_eaes = df_intro_filtered[df_intro_filtered['gtin'].isin(gtins_eaes)]
    df_intro_filtered_foreign = df_intro_filtered[df_intro_filtered['gtin'].isin(gtins_foreign)]
    
    result = (df_intro_filtered_rf['cnt'].sum(), df_intro_filtered_eaes['cnt'].sum(), df_intro_filtered_foreign['cnt'].sum())
    
    return result

In [52]:
# тест
dates = [('2021-01-01', '2021-02-01'),
        ('2021-02-01', '2021-03-01'),
        ('2021-03-01', '2021-03-01'),]
i = 0
import_vs_local(dt1 = '2022-01-01', dt2 = '2022-02-01', tnved = "6D2580183CEF6C8AF1CC72E1C6E6FBC4")

(4825804, 95047, 3495922)

# [-] Определение выраженной сезонности частотным анализом